In [0]:
"""

This cell configures Spark to connect to Azure Data Lake Storage (ADLS) using the storage account access key of the account "noorawitha". After setting the authentication, it lists the directories inside the "lakehouse" container to confirm that the connection to the storage has been successfully established and that the data structure (raw, processed, gold) is accessible.
"""

spark.conf.set(
    "fs.azure.account.key.noorawitha.dfs.core.windows.net",
    "R9RVv+PODzGIJu/ZRXewTS7jkHCtec1N1w9EEe6wb3pSVWfe8L+dKP4Mo+ydLSB50WUQefbGgl1T+AStDcfY6g=="
)

display(
    dbutils.fs.ls("abfss://lakehouse@noorawitha.dfs.core.windows.net/")
)


path,name,size,modificationTime
abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/,gold/,0,1762273074000
abfss://lakehouse@noorawitha.dfs.core.windows.net/processed/,processed/,0,1762269197000
abfss://lakehouse@noorawitha.dfs.core.windows.net/raw/,raw/,0,1759241549000


In [0]:
"""

This cell installs all the required Python libraries used for text processing and feature extraction, including tools for natural language processing, embeddings, readability analysis, and sentiment detection. After installation, a kernel restart is required to ensure that the newly installed packages are properly loaded and can be used in the subsequent cells.
"""

%pip install nltk
%pip install sentence-transformers
%pip install textstat
%pip install textblob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/40.5 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━ 30.7/40.5 kB 741.4 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 748.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.5 MB ? eta -:--:--
   ━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/1.5 MB 10.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━ 1.0/1.5 MB 14.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/803.5 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/78.5 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.8 MB/s eta 0:00:00
Note: you may 

In [0]:
"""

This cell imports all necessary libraries used for Spark-based data processing and NLP feature engineering. It includes Spark SQL functions for data manipulation, Spark ML components for text vectorization and TF-IDF computation, as well as external Python libraries such as NLTK, TextBlob, TextStat, and Sentence Transformers for sentiment analysis, readability scoring, and text embeddings. Additional libraries like NumPy and Pandas are also imported to support numerical operations and general data analysis tasks.
"""

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import (
    col, year, month, dayofweek, to_timestamp,
    length, size, split, udf
)

from pyspark.sql.types import (
    StringType, DoubleType, FloatType, ArrayType
)
from pyspark.ml.feature import (
    Tokenizer, StopWordsRemover, CountVectorizer, IDF
)
from pyspark.ml import Pipeline

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import textstat
from sentence_transformers import SentenceTransformer

import numpy as np
import pandas as pd
import re

from sklearn.feature_extraction.text import TfidfVectorizer


In [0]:
"""
This cell loads the cleaned and curated dataset stored in the Gold layer as a Delta table. After loading, the schema is displayed to verify column names and data types, and a small sample of records is shown to confirm that the data was successfully read and is ready for further processing.
"""

df = spark.read.format("delta").load(
    "abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/curated_reviews_cleaned/"
)

df.printSchema()
display(df.limit(10))


root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- author_name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language: string (nullable = true)
 |-- n_votes: string (nullable = true)
 |-- date_added: date (nullable = true)
 |-- word_count: integer (nullable = true)



review_id,book_id,title,author_id,author_name,user_id,rating,review_text,language,n_votes,date_added,word_count
187e930bad53d51e04b8290a45c8cf96,143223,War in Heaven,3272406,Benoit Attinost,b5f937e40b9cb0e2eecb4ba2365e2019,3,"There were dangling subplots and some holes, but this actually wasn't a bad read.",,107,null,14
fde762c082a8c69defa540f6b728c216,15507958,"Me Before You (Me Before You, #1)",3904122,Ora A. Clement,d6351c177800f63939130f46bb063f44,4,"So the question is, do we let negative life experiences change who we are and determine what we are willing to live for? I'm not sure if we can help it but nothing has made me want to give up on trying more than reading this book. My choice? It's not that simple. But with that being said, the book piqued my interest, made me contemplate the meaning of life, and stirred something inside me that I can't quite pinpoint but have a sneaking suspicion to be associated with previous issues I've had with depression.",eng,51715,2012-12-31,95
c1c87b40244906901d561f5ff6f5e98d,547322,Ten Poems to Last a Lifetime,2845146,Jean-Louis Viot,a9091e712f89280c3012684ec029d2a5,4,"The Art of Disappearing When they say Don't I know you? say no. When they invite you to the party remember what parties are like before answering. Someone telling you in a loud voice they once wrote a poem. Greasy sausage balls on a paper plate. Then reply. If they say We should get together say why? It's not that you don't love them anymore. You're trying to remember something too important to forget. Trees. The monastery bell at twilight. Tell them you have a new project. It will never be finished. When someone recognizes you in a grocery store nod briefly and become a cabbage. When someone you haven't seen in ten years appears at the door, don't start singing him all your new songs. You will never catch up. Walk around feeling like a leaf. Know you could tumble any second. Then decide what to do with your time. This poem is by Naomi Shihab Nye. It was hard to pick my favorite poem from this collection, but this one spoke to me as I flipped back through this book. I have been working my way through this volume of poetry for a few months. Although this is the first volume from Housden that I have posted here, I have enjoyed all of his anthologies. It is wonderful to be able to read his commentaries as well as the hand-picked poems. If you have any interest in poetry, you should at least browse through these collections. Housden has a way of matching up a series of poems from different people and eras that makes all of the selection shine.",,30,null,300
4bfc88504c96093c4d819f9b8b5af003,7640261,Sex at Dawn: The Prehistoric Origins of Modern Sexuality,51906,Bijan B. Gillani,98b87c2d2d96cda894141661036d930b,4,Fascinating look at the history of sexual behavior. Makes one think.,en-US,1411,2010-06-29,11
016acbcc21ec3a003cac585777ed24a6,15507958,"Me Before You (Me Before You, #1)",3904122,Ora A. Clement,fcc708cf0d6c38a0cf2a1b4c374f08c2,5,OMG! Great read. So touching how 1 person can really have an impact if you are open to the possibilities. Impactful!,eng,51715,2012-12-31,21
b58daa3d305709b6f56b12e2ab121254,24572248,Grace's Ground War (Virtues and Valor #5),305163,David V. Loertscher,47a73c38a3acadbd58ad7f55f6e0cde9,5,"Grace's Ground War is the fifth novella in Hallee Bridgeman's Virtues & Valor series. These novellas were written about the World WarII in Europe. The Virtues were the code names for a group of women from different countries used to fight the Nazis. Each Virtue had to remain anonymous to the others so they would not know any information that could be obtained if caught and tortured. But they were still able to meet and pray together. The stories are based on real women and their assignments. Ruth Aubertin was Jewish who converted to Christianity. Her father was a French colonel who trained his children on survivals skills. After his death, Ruth and her brothers Joined the British Resistance. Ruth was given the c

In [0]:
train_df, val_df, test_df = df.randomSplit([0.7, 0.15, 0.15], seed=42)

df = train_df


In [0]:
"""
The dataset is randomly split into three subsets: 70% for training, 15% for validation, and 15% for testing. The count of records in each subset is calculated, and their percentages are printed to ensure the split follows the intended distribution before proceeding to model-related steps.
"""

train, val, test = df.randomSplit([0.7, 0.15, 0.15], seed=42)

train_count = train.count()
val_count = val.count()
test_count = test.count()

total = train_count + val_count + test_count

train_pct = (train_count / total) * 100
val_pct = (val_count / total) * 100
test_pct = (test_count / total) * 100

print(f"Train: {train_pct:.2f}% | Validation: {val_pct:.2f}% | Test: {test_pct:.2f}%")


Train: 70.01% | Validation: 14.99% | Test: 15.00%


In [0]:
"""
Each subset (train, validation, and test) is written back to the Gold layer in Delta format. They are stored in separate directories under feature_v2 to keep the data structure organized and to allow easy reuse during training and evaluation phases.
"""

train.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/feature_v2/train/"
)

val.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/feature_v2/validation/"
)

test.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/feature_v2/test/"
)


In [0]:
"""
Load the training split stored in the Gold layer (feature_v2/train) using Delta format. After loading the dataset, display a small sample to confirm that the data was read correctly and that all expected fields are present. This helps verify that the training portion is ready for the next preprocessing steps.
"""

train = spark.read.format("delta").load(
    "abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/feature_v2/train/"
)

display(train.limit(10))


review_id,book_id,title,author_id,author_name,user_id,rating,review_text,language,n_votes,date_added,word_count
000110a11fc443ebbb21e7ec04969b81,20555443,The Martian,1109155,Deborah Scott,abf6bed26152ae15fd1ddc740f4472d6,5,Loved every moment of it!,eng,802,2014-08-28,5
00011d8803a54715a7c2be5fa751edc1,15034595,"The Devil's Revenge (The Devil's Eyes, #2)",363112,Frazer Hines,73941b527221fe694b3bdb30ed3fd8cd,5,"OhMy wow, just WOW!!! Where do I even begin? Just as awesome as the 1st book!! Action packed & sexy as hell, amazing new charaters (my Fav was Elijah) to go along with what was left of the old crew, lots of twists & surprises, plenty of laughs & soooo damn hard to walk away from my kindle for even a few minutes. Awesome thrill ride!!! God I hope theres more!!!! Love this Author =)",en-US,14,2012-06-04,75
000197dca1e705ed3a3e7be5494e41c7,6185,Wuthering Heights,32327,Simon Lilly,1e5b241799d306b385fc41908b1c11e1,4,"This has to be one of my all time favourite reads, I think it such a passionate and a haunting book. An old teacher and i would spend ages arguing about it as i thought it was so romantic and he would disagree but i stand by it! It has one of the most beautiful lines i've ever read 'i love everything about her, the air above her head and the ground beneath her feet...(or words to that effect) Heathcliff and Cathy loved each other so much it destroyed them, they loved each other in this world and the next....ok when you put it like that he had a point! A great book with some legendary characters.",eng,17961,null,119
000250b46f177fe76441bbd178a64671,2834890,The Dark Tower: The Long Road Home,6511186,Georgina Moseley,7513f8723f2f1b6cbb0646c2bf13af16,4,Amazing graphics. And very cool to read a part of the story I never knew about before. Also made more palatable by the lack of King's involvement. I'm still pissed at him.,eng,217,2008-10-15,32
00027dd587d48a1ea0799f140b0fef60,13459291,Diario de invierno,261384,Bryan Connon,05232449712d38a565f54086c9288757,3,Una biografia escrita como si fuera un bonito paseo por la vida del autor. una manera de ver a traves de los ojos del otro el comienzo Dr una etapa y el fin de muchas otras. Me gusto el libro.,spa,23,2012-02-01,41
00042dd21f48d366b543a1cff86599a1,28239255,La chica de Pablo,6468860,Renea Taylor,fc888c3ddba018ddccfb0b5a96b2af72,2,"Lei este libro por curiosidad mas que por otra cosa, sabia que era una especie de fanfic de Pablo Alboran y eso era lo que mas me llamaba. La historia me ha enganchado porque es facil y rapida de leer, pero en realidad creo que la trama se queda muy floja y que se hace repetitiva.",spa,41,2016-02-09,56
00059fd4ff2a02d8f9ed5e72e2e668c7,2458243,Called to Serve,30571,Shelley Rice,2db93bf461c173314064c2a752db3d66,5,"This book opened my eyes to what I had supposed and believed was going on behind the scenes of what we see on the national stage. James was a special operative in the US Military, is LDS and became an acquaintance of mine shortly before his run for the Presidency against an all-star cast of Bill Clinton, George Bush, and Ross Perot in 1992. I read his book shortly after it was printed in May 1991",eng,1,null,77
00063b1305b5dd27248ce400c4d5636d,292307,Apple Cider Vinegar: Miracle Health System,6989053,Isabella Connor,87da2abd46c55ade4916b3e27dc56b60,4,This book inspired me. It's easy to read and quick. I have been drinking apple cider vinegar every day now; and it stopped my gall bladder attacks.,,33,2005-05-19,27
00065baaf6b19f2ed0c2dd899c9821c4,5639869,أوتار الماء,2901338,Connie Hill,2b6b12abad41f367285dd36fbc9996fa,3,"tnw` qSS~ mmtz lG@ f~ Gy@ ljml qSS nsny@ wjtm`y@ , bl w`lmy@ '`jbn~ `l~ wjh lkhSwS : shrf@ l`Twr lmkhtf~ mrtyn dhlk lwmyD",ara,179,null,30
000671601b00cc04bd8eaf95de8f1ab4,13386741,"The Dark Angels: Tied Together (The Dark Angels, #2)",234255,Janice Monk Glass,9654e72fbdf7aa7f3d9656ae8aeaba72,4,"My copy, provided by the author in exchange for an honest review, is for the second edition from Rocky Ridge Books. Nice pre

In [0]:
"""
Create and apply a text-cleaning function to preprocess the review text before feature extraction. The function standardizes the text by converting it to lowercase, removing URLs, numbers, punctuation, and extra spaces. It also discards extremely short reviews and generates additional columns for word and character counts, preparing the dataset for downstream NLP tasks.
"""

@F.udf(returnType=StringType())
def clean_text(text):
    if not text:
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", " url ", text)
    text = re.sub(r"\d+", " num ", text)
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

clean_train = train.withColumn("clean_review", clean_text(F.col("review_text")))
clean_train = clean_train.filter(F.length("clean_review") > 10)

clean_train = clean_train.withColumn("review_length_words", size(split(col("clean_review"), " "))) \
                   .withColumn("review_length_chars", length(col("clean_review")))

display(clean_train.limit(10))

clean_train.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/feature_v2/clean_train/"
)


review_id,book_id,title,author_id,author_name,user_id,rating,review_text,language,n_votes,date_added,word_count,clean_review,review_length_words,review_length_chars
0000988e8a5143dbce3ed227f4e4a029,17993080,"Shift (Silo, #2)",682719,Alain Beaulieu,fe5909cf487975f99e6ecabc4122df60,5,"Yes I highly recommend to read this book however it is a sequel so you have to read ""Wool"" first to understand this one. It is also a prequel to ""Wool"" telling about the characters that are encountered in it. Sci-fi, Dystopian novel. It took me a really long time to read this because I kept forgetting things that the characters forgot and it's one of the novels that you should really focus all of your attention on. Omnibus book.",eng,29,2013-05-28,82,yes i highly recommend to read this book however it is a sequel so you have to read wool first to understand this one it is also a prequel to wool telling about the characters that are encountered in it scifi dystopian novel it took me a really long time to read this because i kept forgetting things that the characters forgot and its one of the novels that you should really focus all of your attention on omnibus book,80,420
0000aef84f9f5fabd87ef9d885152573,30180620,Bhagavad - Gita,7477727,Kristina Kinderyte,e74bf22cbd182697c4bed57aa6ba2f96,5,"Cand am cautat pe internet sa aflu mai multe despre Bhagavat Gita, am gasit urmatorul raspuns: Q: What can be learned by the study of Bhagavad-Gita? A: Accurate, fundamental knowledge about God, the ultimate truth, creation, birth and death, the results of actions, the eternal soul, liberation and the purpose as well as the goal of human existence. Dupa ce am citit-o recunosc ca asa este: divinitatea se reveleaza mai clar ca niciunde (atat cat e posibil) in aceasta carticica. In Occident, asemenea intuitii originale despre sacru, dar invaluite intr-un nor foarte obscur, am descoperit poate in scrierile lui Meister Eckhart sau in cartulia The Cloud of Unknowing. Pe celebrul Sergiu Al-George, traducatorul volumului si expert orientalist, il cunosc din amintirile din puscariile comuniste relatate de Nicolae Steinhardt in Jurnalul fericirii si, daca imi aduc bine aminte, e pomenit si intr-unul din volumele cartii ,,Inchisoarea noastra cea de toate zilele"" de Ion Ioanid. Recomand cartea celor care s-au plictisit de cotidianul prozaic si limitat.",rum,1,null,169,cand am cautat pe internet sa aflu mai multe despre bhagavat gita am gasit urmatorul raspuns q what can be learned by the study of bhagavadgita a accurate fundamental knowledge about god the ultimate truth creation birth and death the results of actions the eternal soul liberation and the purpose as well as the goal of human existence dupa ce am citito recunosc ca asa este divinitatea se reveleaza mai clar ca niciunde atat cat e posibil in aceasta carticica in occident asemenea intuitii originale despre sacru dar invaluite intrun nor foarte obscur am descoperit poate in scrierile lui meister eckhart sau in cartulia the cloud of unknowing pe celebrul sergiu algeorge traducatorul volumului si expert orientalist il cunosc din amintirile din puscariile comuniste relatate de nicolae steinhardt in jurnalul fericirii si daca imi aduc bine aminte e pomenit si intrunul din volumele cartii inchisoarea noastra cea de toate zilele de ion ioanid recomand cartea celor care sau plictisit de cotidianul prozaic si limitat,164,1020
00022f6f36be151ba7f37bc7db87ce28,33593824,"Lucas (Cold Fury Hockey, #8)",13309148,Lisa Perrin,a0816ef877c36654aa3d6d9a7a00d2a6,4,"this is book number 8 in the Cold Fury hockey series, all can be read as stanadlones but best to read them all to get all there Hottness, hockey selves! Lucas is the newest member of the Cold Fury and be is also fellow player Max' little brother. Lucas and Stephanie make a great couple. I volunteered to read a copy of this book",eng,446,null,67,this is book number num in the cold fury hockey series all can be read as stanadlones but best to read them all to get all ther

In [0]:
from pyspark.sql.functions import col, split, length, size

clean_train = clean_train.withColumn(
    "review_length_words",
    size(split(col("clean_review"), " "))
).withColumn(
    "review_length_chars",
    length(col("clean_review"))
)




IOStream.flush timed out


In [0]:
"""
Load the pre-cleaned training dataset and generate sentiment scores using NLTK’s VADER sentiment analyzer. For each review, compute the positive, neutral, negative, and compound sentiment values. After calculating the scores, preview part of the output to confirm that the sentiment extraction worked correctly before saving the updated dataset back to the Gold layer.
"""

clean_train = spark.read.format("delta").load(
    "abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/feature_v2/clean_train/"
)

nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

def get_sentiment_pos(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['pos']

def get_sentiment_neu(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['neu']

def get_sentiment_neg(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['neg']

def get_sentiment_compound(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['compound']

from pyspark.sql import functions as F

sentiment_train = (
    clean_train
    .withColumn("sentiment_pos", F.udf(get_sentiment_pos, DoubleType())("clean_review"))
    .withColumn("sentiment_neu", F.udf(get_sentiment_neu, DoubleType())("clean_review"))
    .withColumn("sentiment_neg", F.udf(get_sentiment_neg, DoubleType())("clean_review"))
    .withColumn("sentiment_compound", F.udf(get_sentiment_compound, DoubleType())("clean_review"))
)

display(
    sentiment_train.select(
        "clean_review",
        "sentiment_pos",
        "sentiment_neu",
        "sentiment_neg",
        "sentiment_compound"
    ).limit(10)
)

sentiment_train.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/feature_v2/sentiment_train/"
)


[nltk_data] Downloading package vader_lexicon to /home/spark-
[nltk_data]     bb956da6-51b2-492a-91a9-62/nltk_data...


clean_review,sentiment_pos,sentiment_neu,sentiment_neg,sentiment_compound
im not saying that im looking forward to the zombie apocalypse but if that dark day comes i definitely want to make sure im on the good side of cassie forrest sure cassie is a little clumsy or way clumsy if were being honest and the current scariest thing in her life is breaking up with her cad of a boyfriend but within her beats the heart of a heroic and loyal friend who also happens to know how to live off the land when a mysterious illness rages across the country turning the infected into lexers aka zombies cassie and a small group of friends unbearable exboyfriend included make it out of new york just in time and go searching for a safe haven like cassie until the end of the world is much deeper than it may appear on the surface this is not your typical zombie thriller yes there are zombies yes cassie and crew shoot stab behead and generally get gory along the way but the true gems in this book are the moments in between when this small group of survivors must come to terms with their new and horrifying world author sarah lyons fleming does an incredible job of breathing life into the characters of this book the story is filled with beautiful quiet moments we watch james savor his last cigarette ana complain about the lack of shampoo and nelly crack jokes because everyone desperately needs him to through cassies eyes this devastated world is real and so is her longing to keep her friends safe and her wish to see the one she foolishly let get away i loved loved loved this book i laughed i fell head over heels for a character i hated in the beginning of the book i criedtwice this book gripped me i felt like i was part of cassies crew even if it was only in my head i was with them planting tomatoes in the ground digging zombie ditches learning to shoot and always worrying about ifwhen the undead would come,0.221,0.647,0.132,0.9912
lord your heart breaks for ash what augh no spoilers ash has had her heart broken her spirit destroyed she has been hurt and betrayed by everyone she has ever loved and trusted she reminds me sadly of a rescue dog i got she was to be put down the day i came to the shelter she was matted and dirty broken and mistreatedand of course she was the one i picked she was a great dogbut it took forever for her to trust me she was waiting for the kick or the mistreatment ash is alot like phoebe was damaged but so worth the trouble lane is a hottie single parent awesome big brother and son he is a tattoo artist he is not without his scarsand he is not looking for a relationship he has enough on his plate and then they meet and well you need to read itlol i am supposed to be working but i read the blurb and went to peek and boom i was hookedbought it and read in all in a hour and a few minutes it was wonderful heart wrenching sigh worthy even lanes kid is a doll,0.163,0.735,0.102,0.9365
for those who turn on tcm only to bask in the glow of classic actors and actresses entertaining insights and worth picking up for the illustrations alone,0.156,0.779,0.065,0.4215
if youve read the first two its about exactly as good as youd imagine,0.182,0.818,0.0,0.4404
i remember the year exactly because professor elaine hansen gave me and lisa one of my dearest friends ever an a for our writing and presentation on this one,0.211,0.789,0.0,0.7717
i really liked this book bree is great and i love the seniors in her tour group i think i will have to get my own copy of this book since i liked it so much i enjoyed this book just as much the num nd time around and since i now have my own copy i can read it again i really like all the characters in this book it makes me want to travel,0.27,0.73,0.0,0.9689
this book was very eyeopening and disturbing sometimes a bit confusing considering the complicated family trees it helped me understand the mormon history better,0.102,0.705,0.193,-0.3744
i received a copy of this book in exchange for an honest review celia 

In [0]:

"""
Load the sentiment-enhanced dataset and extract TF-IDF features using a Spark ML pipeline. 
The process includes tokenization, stop word removal, term frequency calculation, 
and inverse document frequency weighting. Cache the result, preview the output, 
and save the TF-IDF feature dataset to the Gold layer.
"""

from sklearn.feature_extraction.text import TfidfVectorizer
from pyspark.sql.types import ArrayType, DoubleType
from pyspark.ml.linalg import SparseVector
import numpy as np

sentiment_train = spark.read.format("delta").load(
    "abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/feature_v2/sentiment_train/"
)

sentiment_sample = (
    sentiment_train
        .select("review_id", "clean_review")
        .dropna(subset=["clean_review"])
        .limit(100000)
)

pdf = sentiment_sample.toPandas()
pdf["clean_review"] = pdf["clean_review"].fillna("")

tfidf = TfidfVectorizer(max_features=2000, stop_words="english")
X = tfidf.fit_transform(pdf["clean_review"])   # X is sparse

tfidf_rows = []
for rid, vec in zip(pdf["review_id"], X):
    indices = vec.indices.astype(int)
    values = vec.data.astype(float)
    size = tfidf.max_features
    indices_sorted = np.argsort(indices); sv = SparseVector(size, indices[indices_sorted], values[indices_sorted])
    tfidf_rows.append((rid, sv))

tfidf_spark = spark.createDataFrame(tfidf_rows, ["review_id", "tfidf_features"])

tfidf_train = sentiment_train.join(tfidf_spark, on="review_id", how="inner")

tfidf_train.cache()

display(tfidf_train.select("clean_review", "tfidf_features").limit(10))

tfidf_train.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/feature_v2/tfidf_train/"
)


clean_review,tfidf_features
the cover is pretty to look at but the book isnt exactly what i expected i may browse through it occasionally in the future but for now after one quick read through it will just decorate my bookshelf,"Map(vectorType -> sparse, length -> 2000, indices -> List(172, 339, 552, 565, 677, 899, 923, 1040, 1360, 1401, 1421), values -> List(0.1247957262808905, 0.36615847514951355, 0.369634079767003, 0.3681962037248047, 0.3452059455252664, 0.31032084421546774, 0.18418759312477992, 0.3077071314005847, 0.2984370608287698, 0.350292259551601, 0.14937180015417903))"
lucy is suffering through internet dating for research purposes and do we see the suffering to gather information on internet dating and to find ideas for the victims in her crime novel she suffers through one bad date after another until she meets quinn quinn is an undercover detective looking for the serial killer who is targeting males on the internet dating scene when he meets with lucy he cant fight the attraction even though she may be a cold killer one thing leads to another and lucy falls hard for quinn but when she finds out he thought she could be a killer she is so mad and rightfully so as the book progresses the killer starts sending letters to her favorite author lucy gloating about the murders so even though lucy wants to bbq quinn she turns to him for help who wants to be on the mind of a serial killer anyone who has suffered through internet dating can relate to the nightmares out there from lying jerks to married men the internet dating scene is rampant with the dregs of society but every now and then you hear about the diamond in the rough this was a cute book with funny situations and i loved loved quinn i also am looking forward to reading the rest of the series to get to know lucys writer friends who suffer from manitus it will be interesting to watch them fall in love with all their particular quirks,"Map(vectorType -> sparse, length -> 2000, indices -> List(104, 111, 124, 172, 284, 351, 359, 376, 377, 427, 589, 592, 608, 624, 633, 660, 668, 675, 762, 777, 788, 848, 872, 888, 939, 952, 979, 997, 1042, 1054, 1055, 1065, 1070, 1090, 1110, 1117, 1131, 1219, 1271, 1424, 1450, 1467, 1469, 1495, 1522, 1552, 1554, 1601, 1615, 1656, 1701, 1776, 1783, 1840, 1912, 1918, 1974), values -> List(0.08339162019727421, 0.04640471707076505, 0.055441177930484184, 0.046936664050946895, 0.08209929594614117, 0.07860887099268496, 0.06983555747109464, 0.08267070668964561, 0.4681023055599314, 0.08328382071440178, 0.06816408318158397, 0.08059933657893346, 0.05668098173270039, 0.07438414792852566, 0.06708208150625339, 0.0611208950272979, 0.05864435434725603, 0.06208171390685148, 0.05502925002475187, 0.07752825795355099, 0.05701277557185626, 0.07578490825182206, 0.07113560534767953, 0.04776268339050068, 0.40556373707797894, 0.04347094355690858, 0.08076342924583604, 0.09013373743184185, 0.11338977104801497, 0.036769978937380696, 0.0852014824611118, 0.5019272932566875, 0.08859995040275868, 0.07990456795623076, 0.14476240249547567, 0.06650660750586018, 0.05991415619691931, 0.04891874558345317, 0.07687358660176023, 0.040187468124144345, 0.07862177959155897, 0.07990456795623076, 0.06541062223527087, 0.09060401410581045, 0.14101467039239393, 0.182304540416707, 0.04116059849563992, 0.07950062338862404, 0.07430756134459546, 0.06871485056881618, 0.18199570792380446, 0.05270384842821364, 0.05024205297668111, 0.06690471721074649, 0.12677842489503982, 0.07937754912486235, 0.06994065995917836))"
ummore please num num stars i really really enjoyed this book was it a little predictable for the most part yeah but not completely some things that i really thought were going to happen didnt and i cant even tell you how grateful i am for that i did think that i had it pretty much figured out right until the ending i really liked skye mostly because i completely related to her and all of her social awkwardness of course i loved oliver and all of his teasing and protective ways b

In [0]:
"""
This cell loads the dataset containing the TF-IDF features and then uses a SentenceTransformer model to generate semantic embeddings for each cleaned review. Every review is converted into a dense numerical vector that captures its contextual meaning using a BERT-based architecture. These embedding vectors are stored as a new column in the dataset, enabling more meaningful and context-aware input for downstream machine learning models. After displaying a sample to verify the generated embeddings, the updated dataset is saved back to the Gold layer for use in subsequent modeling stages.
"""
from pyspark.sql.types import ArrayType, FloatType
from sentence_transformers import SentenceTransformer
import numpy as np

tfidf_train = spark.read.format("delta").load(
    "abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/feature_v2/tfidf_train/"
)

text_df = (
    tfidf_train
        .select("review_id", "clean_review")
        .dropna(subset=["clean_review"])
  
)

pdf = text_df.toPandas()
pdf["clean_review"] = pdf["clean_review"].fillna("")

model_name = "all-MiniLM-L6-v2"
model = SentenceTransformer(model_name)

embeddings = model.encode(
    pdf["clean_review"].tolist(),
    batch_size=64,
    show_progress_bar=True
)


emb_rows = [
    (rid, emb.astype(np.float32).tolist())
    for rid, emb in zip(pdf["review_id"], embeddings)
]

embed_df = spark.createDataFrame(
    emb_rows,
    schema=["review_id", "bert_embedding"]
)
embedded_train = tfidf_train.join(embed_df, on="review_id", how="inner")

display(embedded_train.select("clean_review", "bert_embedding").limit(10))


embedded_train.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/feature_v2/embedded_train/"
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

clean_review bert_embedding the cover is pretty to look at but the book isnt exactly what i expected i may browse through it occasionally in the future but for now after one quick read through it will just decorate my bookshelf List(-0.035855475813150406, 0.04883427917957306, 0.02901708334684372, 0.03832710161805153, 0.055223096162080765, 0.033306144177913666, -0.0436878576874733, -0.022935651242733, 0.023539826273918152, 0.0776776596903801, -0.06602827459573746, 0.04513328894972801, 0.004127792548388243, -0.07533050328493118, -0.06233524903655052, -0.09223613888025284, -0.047838300466537476, -0.10535689443349838, 0.037090372294187546, -0.014779578894376755, -0.06089794263243675, 0.011842800304293633, 0.023457353934645653, -0.06025257334113121, -0.050189774483442307, -0.07121411710977554, 0.05122982710599899, 0.02407432347536087, -0.054138898849487305, -0.04066040366888046, -0.037232834845781326, 0.0263229887932539, -0.04349367320537567, 0.022022109478712082, -0.01817965880036354, 0.03923158720135689, 0.030644753947854042, 0.020731138065457344, 0.03663920983672142, 0.00475842272862792, -0.04085525497794151, 0.025875123217701912, 0.003506476292386651, 0.0940382182598114, -0.025280142202973366, 0.016631580889225006, 0.0012589633697643876, -0.03894312307238579, 0.04427596554160118, -0.04946664720773697, 0.031008951365947723, -0.011913469061255455, -0.010618304833769798, -0.0923534706234932, 0.039815548807382584, 0.08357852697372437, -0.02796565368771553, -0.06323380768299103, 0.047577887773513794, -0.03291447460651398, 0.030345337465405464, 0.02734375, -0.01654694601893425, 0.017194513231515884, 0.02487971819937229, 0.030863339081406593, -0.03360635042190552, -0.03161293640732765, -0.027422409504652023, -0.054752837866544724, 0.01607406884431839, 0.04679745063185692, 0.024552060291171074, -0.031895626336336136, 0.012447276152670383, -0.025422586128115654, -0.04957637935876846, -0.07009521871805191, 0.04326115548610687, -0.03425975516438484, -0.047922100871801376, -0.016487715765833855, 0.007380023133009672, 0.04152308404445648, -0.07996071130037308, 0.005573117174208164, 0.06993529945611954, -0.13038215041160583, 0.047202084213495255, 0.019485125318169594, 0.1058378517627716, -0.06541332602500916, -0.038549814373254776, 0.0010001111077144742, -0.10769705474376678, 0.037790801376104355, 0.02496453747153282, 0.0067847962491214275, 0.011078317649662495, 0.040724772959947586, 0.016023745760321617, 0.05225187540054321, 0.08692188560962677, -0.05170148238539696, -0.015270651318132877, -0.14639697968959808, 0.05786304548382759, -0.010450247675180435, -0.07528311759233475, -0.08999289572238922, 0.002287384122610092, -0.02462461218237877, -0.05290867015719414, 0.033542148768901825, 0.011541143991053104, -0.05329487472772598, 0.1406143456697464, 0.00949795264750719, 0.05153824761509895, 0.0665072575211525, 0.09346853941679001, 0.08155250549316406, -0.01178787648677826, 0.051254838705062866, -0.058264367282390594, -0.07647861540317535, 0.007088080979883671, 2.7438875865023472E-33, -0.003627743571996689, 0.041388243436813354, -0.037935249507427216, 0.013089573010802269, 0.0954485684633255, -0.0948217436671257, 0.09202852100133896, 0.0163149181753397, -0.05598064884543419, 0.07538871467113495, 0.01667964644730091, 0.006007758900523186, -0.03318389505147934, 0.07746971398591995, -0.06967335194349289, 0.06365299969911575, -0.10095979273319244, -0.0055391849018633366, -0.09090317040681839, 0.024815905839204788, -0.04316619783639908, 0.022398756816983223, 0.06831394135951996, -0.11365193873643875, -0.03373875841498375, 0.025795336812734604, 0.005129152908921242, 0.006452199071645737, 0.009656558744609356, 0.0335686095058918, -0.024971449747681618, -0.009554505348205566, -0.07182423025369644, -0.08976825326681137, -0.026806361973285675, -0.031149154528975487, -0.06266611814498901, -0.014837125316262245, 0.06576411426067352, 0.012501823715865612, -0.1165471002459526, -0.02050882950425148, -0.004517190158367157, 0.02104032225906849, -0.0145874992012

In [0]:
"""
This cell loads the training dataset that already contains semantic embeddings and computes a readability score for each review using the Flesch Reading Ease metric from the textstat library. The score quantifies how easy or difficult a review is to read based on sentence length and word complexity. The calculated readability value is then added as a new column in the dataset. A sample of the updated data is displayed to confirm that the scores were generated correctly and aligned with the corresponding reviews.
"""

embedded_train = spark.read.format("delta").load(
    "abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/feature_v2/embedded_train/"
)

def compute_readability(text):
    """
    Compute Flesch Reading Ease score for a given review.
    Returns 0.0 if the text is empty or invalid.
    """
    if not text:
        return 0.0
    try:
        return float(textstat.flesch_reading_ease(text))
    except:
        return 0.0

readability_udf = udf(compute_readability, DoubleType())

embedded_train = embedded_train.withColumn(
    "readability_score", readability_udf("clean_review")
)

display(embedded_train.select("clean_review", "readability_score").limit(5))


clean_review,readability_score
the cover is pretty to look at but the book isnt exactly what i expected i may browse through it occasionally in the future but for now after one quick read through it will just decorate my bookshelf,0.0
lucy is suffering through internet dating for research purposes and do we see the suffering to gather information on internet dating and to find ideas for the victims in her crime novel she suffers through one bad date after another until she meets quinn quinn is an undercover detective looking for the serial killer who is targeting males on the internet dating scene when he meets with lucy he cant fight the attraction even though she may be a cold killer one thing leads to another and lucy falls hard for quinn but when she finds out he thought she could be a killer she is so mad and rightfully so as the book progresses the killer starts sending letters to her favorite author lucy gloating about the murders so even though lucy wants to bbq quinn she turns to him for help who wants to be on the mind of a serial killer anyone who has suffered through internet dating can relate to the nightmares out there from lying jerks to married men the internet dating scene is rampant with the dregs of society but every now and then you hear about the diamond in the rough this was a cute book with funny situations and i loved loved quinn i also am looking forward to reading the rest of the series to get to know lucys writer friends who suffer from manitus it will be interesting to watch them fall in love with all their particular quirks,0.0
ummore please num num stars i really really enjoyed this book was it a little predictable for the most part yeah but not completely some things that i really thought were going to happen didnt and i cant even tell you how grateful i am for that i did think that i had it pretty much figured out right until the ending i really liked skye mostly because i completely related to her and all of her social awkwardness of course i loved oliver and all of his teasing and protective ways but i also really liked patrick which made it hard to root for anyone it kind of sucks when there is no clear cut bad guy of course its pretty obvious who skye belongs with so theres that i cant wait to find out what happens next i also hope that we get to read about more about bridget in the future because she was a fun character as well overall this was a sweet and fun read and i enjoyed the writing style and feel like all of the characters were welldeveloped and frankly i cant wait to read more i received an arc from the authorpublisher in exchange for an honest review,0.0
im a shark as usual i am two ages and at least half a worldending cataclysm behind the bandwagon but i must say nimona was largely worth the wait i had seen panels floating around the internet and had it recommended to me endlessly but due to a combination of unfamiliarity with graphic novels and the places that stock them and being more easily distracted than a kitten on the plane of shiny fluffy things i never found it either during its incarnation as a webcomic or in print on my own as the screenshotted online panels and art style suggest nimona is a fun pacy relatively uncomplicated webcomic about the titular shapeshifting menace and her beleagured boss taking on the corrupt institution of law enforcement and heroics and leaving a merry trail of destruction as they get to the bottom of a sinister poisoning plot the story itself isnt really all that influential and theres not a lot of time spent setting up how and why the world operates as it does but its quite obvious that that is not the point of nor terribly important to the real point of the story the heart of the story is about the characters though funnily enough i dont think it was all that much about nimona herself the reader spends most of their time with her boss blackheart working through his bizrrely unvillainous mindset and following up on the trauma that resulted

In [0]:
"""
This cell computes the subjectivity score for each review using TextBlob’s sentiment analysis functionality. The subjectivity score measures how opinion-based or emotionally driven the text is, ranging from 0 (completely objective and factual) to 1 (highly subjective and opinion-focused). After calculating the score for every review, the result is stored as a new column in the dataset. A small sample of the updated dataset is then displayed to verify that the subjectivity values were generated correctly.
"""

def compute_subjectivity(text):
    """
    Compute subjectivity score using TextBlob.
    Returns 0.0 for empty or invalid text.
    """
    if not text:
        return 0.0
    return float(TextBlob(text).sentiment.subjectivity)

subjectivity_udf = udf(compute_subjectivity, DoubleType())

embedded_train = embedded_train.withColumn(
    "subjectivity", subjectivity_udf("clean_review")
)

display(embedded_train.select("clean_review", "subjectivity").limit(5))


clean_review,subjectivity
the cover is pretty to look at but the book isnt exactly what i expected i may browse through it occasionally in the future but for now after one quick read through it will just decorate my bookshelf,0.39999999999999997
lucy is suffering through internet dating for research purposes and do we see the suffering to gather information on internet dating and to find ideas for the victims in her crime novel she suffers through one bad date after another until she meets quinn quinn is an undercover detective looking for the serial killer who is targeting males on the internet dating scene when he meets with lucy he cant fight the attraction even though she may be a cold killer one thing leads to another and lucy falls hard for quinn but when she finds out he thought she could be a killer she is so mad and rightfully so as the book progresses the killer starts sending letters to her favorite author lucy gloating about the murders so even though lucy wants to bbq quinn she turns to him for help who wants to be on the mind of a serial killer anyone who has suffered through internet dating can relate to the nightmares out there from lying jerks to married men the internet dating scene is rampant with the dregs of society but every now and then you hear about the diamond in the rough this was a cute book with funny situations and i loved loved quinn i also am looking forward to reading the rest of the series to get to know lucys writer friends who suffer from manitus it will be interesting to watch them fall in love with all their particular quirks,0.6348039215686275
ummore please num num stars i really really enjoyed this book was it a little predictable for the most part yeah but not completely some things that i really thought were going to happen didnt and i cant even tell you how grateful i am for that i did think that i had it pretty much figured out right until the ending i really liked skye mostly because i completely related to her and all of her social awkwardness of course i loved oliver and all of his teasing and protective ways but i also really liked patrick which made it hard to root for anyone it kind of sucks when there is no clear cut bad guy of course its pretty obvious who skye belongs with so theres that i cant wait to find out what happens next i also hope that we get to read about more about bridget in the future because she was a fun character as well overall this was a sweet and fun read and i enjoyed the writing style and feel like all of the characters were welldeveloped and frankly i cant wait to read more i received an arc from the authorpublisher in exchange for an honest review,0.499032738095238
im a shark as usual i am two ages and at least half a worldending cataclysm behind the bandwagon but i must say nimona was largely worth the wait i had seen panels floating around the internet and had it recommended to me endlessly but due to a combination of unfamiliarity with graphic novels and the places that stock them and being more easily distracted than a kitten on the plane of shiny fluffy things i never found it either during its incarnation as a webcomic or in print on my own as the screenshotted online panels and art style suggest nimona is a fun pacy relatively uncomplicated webcomic about the titular shapeshifting menace and her beleagured boss taking on the corrupt institution of law enforcement and heroics and leaving a merry trail of destruction as they get to the bottom of a sinister poisoning plot the story itself isnt really all that influential and theres not a lot of time spent setting up how and why the world operates as it does but its quite obvious that that is not the point of nor terribly important to the real point of the story the heart of the story is about the characters though funnily enough i dont think it was all that much about nimona herself the reader spends most of their time with her boss blackheart working through his bizrrely unvillainous mindset and 

In [0]:
"""
Compute the average word length for each review to get an indication of writing complexity. Reviews containing longer words typically reflect more advanced language, while shorter words suggest simpler phrasing. After calculating the average length for every review, store the value as a new column in the dataset and preview a small sample to verify that the metric was added correctly.
"""

def compute_avg_word_length(text):
    """
    Calculate average word length for a given review.
    Returns 0.0 for empty text.
    """
    if not text:
        return 0.0
    words = text.split()
    return float(np.mean([len(w) for w in words])) if words else 0.0

avg_word_len_udf = udf(compute_avg_word_length, DoubleType())

embedded_train = embedded_train.withColumn(
    "avg_word_length", avg_word_len_udf("clean_review")
)

display(embedded_train.select("clean_review", "avg_word_length").limit(5))


clean_review,avg_word_length
the cover is pretty to look at but the book isnt exactly what i expected i may browse through it occasionally in the future but for now after one quick read through it will just decorate my bookshelf,4.2631578947368425
lucy is suffering through internet dating for research purposes and do we see the suffering to gather information on internet dating and to find ideas for the victims in her crime novel she suffers through one bad date after another until she meets quinn quinn is an undercover detective looking for the serial killer who is targeting males on the internet dating scene when he meets with lucy he cant fight the attraction even though she may be a cold killer one thing leads to another and lucy falls hard for quinn but when she finds out he thought she could be a killer she is so mad and rightfully so as the book progresses the killer starts sending letters to her favorite author lucy gloating about the murders so even though lucy wants to bbq quinn she turns to him for help who wants to be on the mind of a serial killer anyone who has suffered through internet dating can relate to the nightmares out there from lying jerks to married men the internet dating scene is rampant with the dregs of society but every now and then you hear about the diamond in the rough this was a cute book with funny situations and i loved loved quinn i also am looking forward to reading the rest of the series to get to know lucys writer friends who suffer from manitus it will be interesting to watch them fall in love with all their particular quirks,4.397590361445783
ummore please num num stars i really really enjoyed this book was it a little predictable for the most part yeah but not completely some things that i really thought were going to happen didnt and i cant even tell you how grateful i am for that i did think that i had it pretty much figured out right until the ending i really liked skye mostly because i completely related to her and all of her social awkwardness of course i loved oliver and all of his teasing and protective ways but i also really liked patrick which made it hard to root for anyone it kind of sucks when there is no clear cut bad guy of course its pretty obvious who skye belongs with so theres that i cant wait to find out what happens next i also hope that we get to read about more about bridget in the future because she was a fun character as well overall this was a sweet and fun read and i enjoyed the writing style and feel like all of the characters were welldeveloped and frankly i cant wait to read more i received an arc from the authorpublisher in exchange for an honest review,4.155339805825243
im a shark as usual i am two ages and at least half a worldending cataclysm behind the bandwagon but i must say nimona was largely worth the wait i had seen panels floating around the internet and had it recommended to me endlessly but due to a combination of unfamiliarity with graphic novels and the places that stock them and being more easily distracted than a kitten on the plane of shiny fluffy things i never found it either during its incarnation as a webcomic or in print on my own as the screenshotted online panels and art style suggest nimona is a fun pacy relatively uncomplicated webcomic about the titular shapeshifting menace and her beleagured boss taking on the corrupt institution of law enforcement and heroics and leaving a merry trail of destruction as they get to the bottom of a sinister poisoning plot the story itself isnt really all that influential and theres not a lot of time spent setting up how and why the world operates as it does but its quite obvious that that is not the point of nor terribly important to the real point of the story the heart of the story is about the characters though funnily enough i dont think it was all that much about nimona herself the reader spends most of their time with her boss blackheart working through his bizrrely unvillainous mindset and

In [0]:
"""
Calculate the lexical diversity for each review to estimate how rich and varied the vocabulary is. The metric is computed by dividing the number of unique words by the total word count, where higher values reflect more diverse language use. After generating the score for every review, add it as a new column to the dataset and display a small sample to ensure the values were computed correctly.
"""

def compute_lexical_diversity(text):
    """
    Compute lexical diversity for a given review.
    Returns 0.0 for empty text.
    """
    if not text:
        return 0.0
    words = text.split()
    unique_words = set(words)
    return float(len(unique_words) / len(words)) if words else 0.0

lexical_div_udf = udf(compute_lexical_diversity, DoubleType())

embedded_train = embedded_train.withColumn(
    "lexical_diversity", lexical_div_udf("clean_review")
)

display(embedded_train.select("clean_review", "lexical_diversity").limit(5))


clean_review,lexical_diversity
the cover is pretty to look at but the book isnt exactly what i expected i may browse through it occasionally in the future but for now after one quick read through it will just decorate my bookshelf,0.8421052631578947
lucy is suffering through internet dating for research purposes and do we see the suffering to gather information on internet dating and to find ideas for the victims in her crime novel she suffers through one bad date after another until she meets quinn quinn is an undercover detective looking for the serial killer who is targeting males on the internet dating scene when he meets with lucy he cant fight the attraction even though she may be a cold killer one thing leads to another and lucy falls hard for quinn but when she finds out he thought she could be a killer she is so mad and rightfully so as the book progresses the killer starts sending letters to her favorite author lucy gloating about the murders so even though lucy wants to bbq quinn she turns to him for help who wants to be on the mind of a serial killer anyone who has suffered through internet dating can relate to the nightmares out there from lying jerks to married men the internet dating scene is rampant with the dregs of society but every now and then you hear about the diamond in the rough this was a cute book with funny situations and i loved loved quinn i also am looking forward to reading the rest of the series to get to know lucys writer friends who suffer from manitus it will be interesting to watch them fall in love with all their particular quirks,0.5622489959839357
ummore please num num stars i really really enjoyed this book was it a little predictable for the most part yeah but not completely some things that i really thought were going to happen didnt and i cant even tell you how grateful i am for that i did think that i had it pretty much figured out right until the ending i really liked skye mostly because i completely related to her and all of her social awkwardness of course i loved oliver and all of his teasing and protective ways but i also really liked patrick which made it hard to root for anyone it kind of sucks when there is no clear cut bad guy of course its pretty obvious who skye belongs with so theres that i cant wait to find out what happens next i also hope that we get to read about more about bridget in the future because she was a fun character as well overall this was a sweet and fun read and i enjoyed the writing style and feel like all of the characters were welldeveloped and frankly i cant wait to read more i received an arc from the authorpublisher in exchange for an honest review,0.6116504854368932
im a shark as usual i am two ages and at least half a worldending cataclysm behind the bandwagon but i must say nimona was largely worth the wait i had seen panels floating around the internet and had it recommended to me endlessly but due to a combination of unfamiliarity with graphic novels and the places that stock them and being more easily distracted than a kitten on the plane of shiny fluffy things i never found it either during its incarnation as a webcomic or in print on my own as the screenshotted online panels and art style suggest nimona is a fun pacy relatively uncomplicated webcomic about the titular shapeshifting menace and her beleagured boss taking on the corrupt institution of law enforcement and heroics and leaving a merry trail of destruction as they get to the bottom of a sinister poisoning plot the story itself isnt really all that influential and theres not a lot of time spent setting up how and why the world operates as it does but its quite obvious that that is not the point of nor terribly important to the real point of the story the heart of the story is about the characters though funnily enough i dont think it was all that much about nimona herself the reader spends most of their time with her boss blackheart working through his bizrrely unvillainous mindset

In [0]:
"""
Load the embedding enhanced dataset from the Gold layer and store it as a unified training table. This step merges all engineered features including readability, subjectivity, lexical diversity, embeddings, and other attributes into a single Delta dataset. Consolidating everything in one place makes it easier to access and reuse the final feature set during model development or later analysis.
"""

embedded_train = spark.read.format("delta").load(
    "abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/feature_v2/embedded_train/"
)

combined_train = embedded_train
combined_train.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save("abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/feature_v2/combined_train/")


In [0]:
display(combined_train.limit(10))

review_id book_id title author_id author_name user_id rating review_text language n_votes date_added word_count clean_review review_length_words review_length_chars sentiment_pos sentiment_neu sentiment_neg sentiment_compound tfidf_features bert_embedding 0000018556333fe90750481ea7cdc79f 8660865 Encyclopedia of the Exquisite: An Anecdotal History of Elegant Delights 6453478 Cecile Lawrence and Natalie Churn d8e7a064f474ef2a1253042b794a1432 2 The cover is pretty to look at , but the book isn't exactly what I expected. I may browse through it occasionally in the future, but for now after one quick read through, it will just decorate my bookshelf. en-GB 70 2010-11-02 39 the cover is pretty to look at but the book isnt exactly what i expected i may browse through it occasionally in the future but for now after one quick read through it will just decorate my bookshelf 38 199 0.057 0.943 0.0 0.2732 Map(vectorType -> sparse, length -> 2000, indices -> List(172, 339, 552, 565, 677, 899, 923, 1040, 1360, 1401, 1421), values -> List(0.1247957262808905, 0.36615847514951355, 0.369634079767003, 0.3681962037248047, 0.3452059455252664, 0.31032084421546774, 0.18418759312477992, 0.3077071314005847, 0.2984370608287698, 0.350292259551601, 0.14937180015417903)) List(-0.035855475813150406, 0.04883427917957306, 0.02901708334684372, 0.03832710161805153, 0.055223096162080765, 0.033306144177913666, -0.0436878576874733, -0.022935651242733, 0.023539826273918152, 0.0776776596903801, -0.06602827459573746, 0.04513328894972801, 0.004127792548388243, -0.07533050328493118, -0.06233524903655052, -0.09223613888025284, -0.047838300466537476, -0.10535689443349838, 0.037090372294187546, -0.014779578894376755, -0.06089794263243675, 0.011842800304293633, 0.023457353934645653, -0.06025257334113121, -0.050189774483442307, -0.07121411710977554, 0.05122982710599899, 0.02407432347536087, -0.054138898849487305, -0.04066040366888046, -0.037232834845781326, 0.0263229887932539, -0.04349367320537567, 0.022022109478712082, -0.01817965880036354, 0.03923158720135689, 0.030644753947854042, 0.020731138065457344, 0.03663920983672142, 0.00475842272862792, -0.04085525497794151, 0.025875123217701912, 0.003506476292386651, 0.0940382182598114, -0.025280142202973366, 0.016631580889225006, 0.0012589633697643876, -0.03894312307238579, 0.04427596554160118, -0.04946664720773697, 0.031008951365947723, -0.011913469061255455, -0.010618304833769798, -0.0923534706234932, 0.039815548807382584, 0.08357852697372437, -0.02796565368771553, -0.06323380768299103, 0.047577887773513794, -0.03291447460651398, 0.030345337465405464, 0.02734375, -0.01654694601893425, 0.017194513231515884, 0.02487971819937229, 0.030863339081406593, -0.03360635042190552, -0.03161293640732765, -0.027422409504652023, -0.054752837866544724, 0.01607406884431839, 0.04679745063185692, 0.024552060291171074, -0.031895626336336136, 0.012447276152670383, -0.025422586128115654, -0.04957637935876846, -0.07009521871805191, 0.04326115548610687, -0.03425975516438484, -0.047922100871801376, -0.016487715765833855, 0.007380023133009672, 0.04152308404445648, -0.07996071130037308, 0.005573117174208164, 0.06993529945611954, -0.13038215041160583, 0.047202084213495255, 0.019485125318169594, 0.1058378517627716, -0.06541332602500916, -0.038549814373254776, 0.0010001111077144742, -0.10769705474376678, 0.037790801376104355, 0.02496453747153282, 0.0067847962491214275, 0.011078317649662495, 0.040724772959947586, 0.016023745760321617, 0.05225187540054321, 0.08692188560962677, -0.05170148238539696, -0.015270651318132877, -0.14639697968959808, 0.05786304548382759, -0.010450247675180435, -0.07528311759233475, -0.08999289572238922, 0.002287384122610092, -0.02462461218237877, -0.05290867015719414, 0.033542148768901825, 0.011541143991053104, -0.05329487472772598, 0.1406143456697464, 0.00949795264750719, 0.05153824761509895, 0.0665072575211525, 0.09346853941679001, 0.08155250549316406, -0.01178787648677826, 0.051254838705062866, -0.058264367282390594, -0.076478615

In [0]:
final_train = embedded_train

final_train.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save("abfss://lakehouse@noorawitha.dfs.core.windows.net/gold/features_v2/final_train")
